# Tearing the Mask off Python Decorators
## Deep look into some advanced concepts
<img src='images/jelly.jpg'></img>
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@infonautica?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Leonid Danilov</a>
        on 
        <a href='https://www.pexels.com/photo/photo-of-jellyfish-2690765/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels</a>
    </strong>
</figcaption>

### Introduction

### Functions Are Objects

### Scope

### Closure

### Finally, Decorators